## Install necessary Python packages

In [1]:
%pip install -U crewai crewai_tools load_dotenv langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [2]:
from crewai_tools import YoutubeChannelSearchTool

yt_tool = YoutubeChannelSearchTool(youtube_channel_handle='@3blue1brown')

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]2024-10-27 16:18:02,215 - 123916539007104 - local_persistent_hnsw.py-local_persistent_hnsw:271 - WARNING: Add of existing embedding ID: default-app-id--416a2f2f2c70cf36119f4e4b1f75f1a39779ba68eb52a363208b5f150ba5097d
2024-10-27 16:18:02,218 - 123916539007104 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--416a2f2f2c70cf36119f4e4b1f75f1a39779ba68eb52a363208b5f150ba5097d
2024-10-27 16:18:02,219 - 123916539007104 - local_persistent_hnsw.py-local_persistent_hnsw:271 - WARNING: Add of existing embedding ID: default-app-id--416a2f2f2c70cf36119f4e4b1f75f1a39779ba68eb52a363208b5f150ba5097d
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.82it/s]


In [3]:
from crewai import Agent
# from crewai_tools import YoutubeChannelSearchTool
import os
from dotenv import load_dotenv

load_dotenv()

# os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
# os.environ["OPENAI_MODEL_NAME"] = "gpt-4-0125-preview"



True

In [ ]:


##senior blog contect reserchers
blog_researcher = Agent(
    role='Blog Reserachers from youtube videos',
    goal = 'Get the relevant video content for the topic: {topic} from youtube channel',
    name='blog_researchers',
    description='Get the relevant video content for the topic: {topic} from youtube channel',
    verbose=True,
    memory = True,
    backstory = (
            "Expert in understanding videos in AI Data Scinece, machine learning and gen ai and providing suggestions"
    ),
    tools=[yt_tool],
    allow_delegation=True
)

## creating a senior blog writer agent with yt tool

blog_writer = Agent(
    role = 'Blog Writer',
    goal = 'Narrate compelling tech stories about the video: {topic} from youtube channel',
    verbose=True,
    memory = True,
    backstory = (
            "Expert in simplifying complex topics, you craft engaging narratives that educate and bring new discoveries to \
                light in an accessible manner"
    ),
    tools=[yt_tool],
    allow_delegation=False
)

In [21]:
from crewai import Task

##Reserach task
research_task = Task(
    name="Research",
    description="Identify the video: {topic}. Get the detailed information about the video from the channel.",
    expected_output = "A comprehensive 3 paragraphs long report based on the {topic} of video ",
    tools=[yt_tool],
    agent=blog_researcher,
)

## Writing task with language model configuration
write_task = Task(
    description=(
        "Get the info from the youtube channel on the topic: {topic}."
    ),
    expected_output="Summarize the info from the youtube channel video on the topic: {topic} and create the content for the blog",
    tools=[yt_tool],
    agent=blog_writer,
    async_execution = False,
    output_file="new-blog-post.md"
)

In [23]:
from crewai import Crew, Process


crew = Crew(
    agents = [blog_researcher, blog_writer],
    tasks = [research_task, write_task],
    process = Process.sequential,
    memory = True,
    cache = True,
    max_rpm = 100,
    share_crew=True
)

##start the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'But what is a Fourier series? From heat flow to drawing with'})
print(result)

2024-10-27 16:10:47,335 - 137415314755712 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Blog Reserachers from youtube videos
## Task: Identify the video: But what is a Fourier series? From heat flow to drawing with. Get the detailed information about the video from the channel.


2024-10-27 16:11:10,488 - 137415314755712 - local_persistent_hnsw.py-local_persistent_hnsw:347 - WARNING: Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1




# Agent: Blog Reserachers from youtube videos
## Thought: Thought: To find the specific video titled "But what is a Fourier series? From heat flow to drawing with" from the 3blue1brown YouTube channel, I need to use the tool at my disposal to search the channel's content with the exact or closely matching video title.
## Using tool: Search a Youtube Channels content
## Tool Input: 
"{\"search_query\": \"But what is a Fourier series? From heat flow to drawing with\"}"
## Tool Output: 
Relevant Content:
@3blue1brown


# Agent: Blog Reserachers from youtube videos
## Final Answer: 
I apologize for the confusion, but it seems there was an error in retrieving the specific details from the action performed. Without the actual content retrieved from the search tool, I'm unable to provide the comprehensive 3 paragraphs long report based on the "But what is a Fourier series? From heat flow to drawing with" video from the 3blue1brown channel as was requested.


# Agent: Blog Writer
## Task: Ge

## Different example to check the workflow

In [4]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry, based on the research analyst’s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)

# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
crew.kickoff()


 
[2024-10-27 16:18:45][INFO]: Planning the crew execution
# Agent: Market Research Analyst
## Task: Research the latest trends in the AI industry and provide a summary.Conduct research on the current trends, technologies, and key players in the AI industry.


# Agent: Market Research Analyst
## Thought: I need to gather information on the latest trends, technologies, and key players in the AI industry to summarize the top 3 trending developments along with their significance.
## Using tool: Search the internet
## Tool Input: 
"{\"name\": \"AI Industry Trends\", \"description\": \"Research the latest trends in the AI industry, including technologies, key players, and developments\", \"args_schema\": {}, \"return_direct\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Market Research Analyst
## Thought: Thought: I need to search for the latest trends in the AI industry to gather information about technologies, key players, and developm

CrewOutput(raw="```markdown\n# The Transformative Trends in the AI Industry: A 2023 Overview\n\nIn 2023, the artificial intelligence (AI) industry is undergoing significant transformations that are reshaping various sectors. With advancements in technology, ethical considerations, and healthcare innovations, AI is moving towards a future that not only enhances productivity but also aims to align with societal values. This post will delve into three prominent trends that are defining the current landscape of AI.\n\n## Generative AI: Revolutionizing Creativity\n\nOne of the most exciting developments in the AI realm is the rise of generative AI models, such as OpenAI's ChatGPT and DALL-E. These sophisticated systems can now produce human-like text, art, and even music, revolutionizing the creative process. Businesses are leveraging these technologies to enhance productivity and innovate customer engagement strategies. Sectors like marketing and entertainment are rapidly adapting to the c